In [1]:
import pandas as pd

In [2]:
from keys import keys
from binance.client import Client

In [3]:
client = Client(api_key=keys.apiKey, api_secret=keys.secretKey)

In [9]:
"""
The various columns used by Binance
taken from:
https://python-binance.readthedocs.io/en/latest/binance.html#module-binance.client
"""

binance_columns = ['Open Time', 'Open', 'High', 'Low', 'Close',
            'Volume', 'Close Time', 'Quote asset volume', 
            'n_trades', 'Taker buy base asset volume', 
            'Taker buy quote asset volume', 'Ignore']

In [10]:
"""
The various time intervals binance provides
"""

INTERVAL_1MIN = Client.KLINE_INTERVAL_1MINUTE
INTERVAL_5MIN = Client.KLINE_INTERVAL_5MINUTE
INTERVAL_15MIN = Client.KLINE_INTERVAL_15MINUTE
INTERVAL_30MIN = Client.KLINE_INTERVAL_30MINUTE

In [6]:
"""
DOWNLOADS DATA FROM BINANCE IN BATCHES OF 500 items.
In total will return 500*n_rows - (n_rows+1) and
no duplicate data should be present

INPUTS:
    n_iterations: how many batches of 500 items to download
    
    currency: one of the varios currencies, examples: 
                'BTCUSDT', 'LTCUSDT' ...etc
    interval:   Client.KLINE_INTERVAL_1MINUTE
                Client.KLINE_INTERVAL_3MINUTE
                Client.KLINE_INTERVAL_5MINUTE
                Client.KLINE_INTERVAL_15MINUTE
                Client.KLINE_INTERVAL_30MINUTE
                Client.KLINE_INTERVAL_1HOUR
                ...
"""
def get_data_by_intervals(n_iterations, currencies, interval):
    
    #most recent timestamp to start downloading from
    starting_point = client.get_klines(symbol=currencies[0], 
                                  interval=interval, limit=1)[-1][0]
    
    main_df = pd.DataFrame()
    
    for currency in currencies:
        currency_cols = []
        for col in binance_columns:
            currency_cols.append(currency+' '+col)
        specific_currency_df = pd.DataFrame(columns=currency_cols)
        
        #downloading n iterations of a specific currency
        for i in range(0,n_iterations):
            #split in if elseto use starting_point
            if specific_currency_df.empty:
                new_data = client.get_klines(symbol=currency, 
                                              interval=interval,
                                              endTime = starting_point,
                                              limit=500
                                            )
            else:
                end_time = specific_currency_df[currency+' Open Time'].iloc[0]
                new_data = client.get_klines(symbol=currency, 
                                              interval=interval,
                                              endTime=end_time,
                                              limit=500
                                            )
            #merging all the rows of a specific currency in one DataFrame
            #we need to delete the first row of specific_currency_df or
            # delete the last row of new_data_df, it's repeated
            new_data_df = pd.DataFrame(new_data, columns=currency_cols)
            specific_currency_df = pd.concat([new_data_df,specific_currency_df[1:]],
                                             ignore_index=True)
            
        #merging the specific currency created into the main_df were we keep all the data
        main_df = pd.concat([main_df, specific_currency_df],axis=1)
    return main_df